# Getting Standalone BOB (sBOB) Waveforms using NRPy+

### Author: Siddharth Mahesh

In this tutorial, we document the standalone Backwards One Body model for the late merger-ringdown phase of Binary Black Hole mergers. 

## Importing Core Modules

We first import the necessary Nrpy+ and python modules in order to build the waveform 

In [1]:
# Step 1: Initialize core Python/NRPy+ modules
import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
from outputC import outputC,outCfunction # NRPy+: Core C code output module
import sympy as sp                       # SymPy: The Python computer algebra package upon which NRPy+ depends

## Initialize input parameters and variables

The standalone BOB model has two input parameters, the final mass and final spin. The first quantities we compute from them are the ISCO orbital frequency and the QNM frequency. The second is the peak strain, which can be matched to the peak strain of any waveform approximant. The third and fourth are the frequency and damping time of the $2,2$ Quasinormal Mode (QNM) of the merged black hole.

We declare these below as sympy symbols. Furthermore, the model's expressions are more concisely given by the following reductions:

$$
\Omega^4_\pm = \frac{\Omega_{\rm{QNM}} \pm \Omega_0}{2}
$$

Lastly, in this module, we wish to implement a time-domain sBOB waveform and will therefore define the time symbol to use as an argument in future computations. 

In [2]:
t = sp.Symbol("t")
Mf, af = sp.symbols('Mf af')

## Fits From Sean's Code
Mhalf = 1 - (1 - Mf)/2
Omega_0 = (-0.091933*af + 0.097593)/(af*af - 2.4228*af + 1.4366)
omega_QNM = (1.5251-1.1568*(1-af)**0.1292)/Mf

##
Omega_QNM = omega_QNM/2
Omega4_plus , Omega4_minus = (Omega_QNM**4 + Omega_0**4)/2 , (Omega_QNM**4 - Omega_0**4)/2

## Define the QNM damping time

The QNM damping time $\tau$ is given by

$$
\tau = \frac{Q}{\omega_{\rm QNM}}
$$

In [3]:
Q = 0.7+1.4187*(1-af)**(-0.499)
tau = Q/omega_QNM

## Define the peak $\psi_4$ time 

The standalone BOB model sets the time of the peak strain amplitude to $t_{p,strain} = 0$. However, model quantities are derived based on the time of the peak of the Weyl scalar $\psi_4$ which we will denote as $t_p$. This is given in terms of the input parameters as:

$$
t_p = -\tau\log\left(\frac{\Omega_0}{\Omega_{QNM}}\right)
$$


In [4]:
t_p = -1*tau*sp.log(Omega_0/Omega_QNM)

## Define the Orbital Frequency evolution

Now that we have defined the peak $\psi_4$ time, we can define the evolution of the orbital frequency as a function of time as follows:

$$
\Omega = \left\{\Omega_+^4 + \Omega_-^4\tanh\left(\frac{t - t_p}{\tau}\right)\right\}^{1/4}
$$

In [5]:
Omega = (Omega4_plus + Omega4_minus*sp.tanh((t - t_p)/tau))**sp.Rational(1,4)
omega = 2*Omega

## Define the Orbital Phase evolution

The orbital phase is given as follows:

\begin{align}
\Phi = \tau \left( \kappa_-\left(\tan^{-1}\left(\frac{\Omega}{\kappa_-}\right) - \tan^{-1}\left(\frac{\Omega_+}{\kappa_-}\right)\right)\\
\quad\quad + \kappa_-\left(\tanh^{-1}\left(\frac{\Omega}{\kappa_-}\right) - \tanh^{-1}\left(\frac{\Omega_+}{\kappa_-}\right)\right)\\
\quad\quad - \kappa_+\left(\tan^{-1}\left(\frac{\Omega}{\kappa_+}\right) - \tan^{-1}\left(\frac{\Omega_+}{\kappa_+}\right)\right)\\
\quad\quad- \kappa_+\left(\tanh^{-1}\left(\frac{\Omega}{\kappa_+}\right) - \tanh^{-1}\left(\frac{\Omega_+}{\kappa_+}\right)\right) \right)
\end{align} 

Where the terms $\kappa_\pm$ are given by:

$$
\kappa_+ = \Omega_0\\
\kappa_- = \Omega_{QNM}
$$

Also, to avoid domain errors in the inverse hyperbolic arctangent, we introduce the alternative form:

$$
\tanh^{-1}x = \frac{1}{2}\ln\left(\frac{1 + x}{1 - x}\right)
$$

Finally, in the quasicircular case, the waveform phase is given by:

$$
\varphi = 2\Phi
$$

In [6]:
## Compute Phase:
## We use here the alternative definition of arctan
## arctan(x) = 0.5*ln( (1+x)/(1-x) )
Omega_plus = Omega4_plus**sp.Rational(1,4)
arctanhp = sp.Rational(1,2)*Omega_QNM*tau*sp.log( (1 + Omega/Omega_QNM)*(1 - Omega_plus/Omega_QNM) / ( (1 - Omega/Omega_QNM)*(1 + Omega_plus/Omega_QNM) ) )
arctanhm = sp.Rational(1,2)*Omega_0*tau*sp.log( (1 + Omega/Omega_0)*(1 - Omega_plus/Omega_0) / ( (1 - Omega/Omega_0)*(1 + Omega_plus/Omega_0) ) )
arctanp = Omega_QNM*tau*( sp.atan(Omega/Omega_QNM) - sp.atan(Omega_plus/Omega_QNM) )
arctanm = Omega_0*tau*( sp.atan(Omega/Omega_0) - sp.atan(Omega_plus/Omega_0) )

Phi = arctanhp+arctanp-arctanhm-arctanm

phase = 2*Phi

## Define the Strain Amplitude evolution

In order to define the strain amplitude, we first define the BOB $\psi_4$ amplitude as:

$$
|\psi_4| = A_p\mathrm{sech}\left(\frac{t - t_p}{\tau}\right)
$$

Where $A_p$ is the peak value of $\psi_4$. We then apply the quasicircular approximation to get the strain amplitude:

$$
|h| = \frac{A_p}{\omega^2}\mathrm{sech}\left(\frac{t - t_p}{\tau}\right)
$$
We also use the normalization $M_{1/2}/M_f$ to account for the loss in total energy due to radiation.
We use a fit for $A_p$.

In [7]:
A_p = 0.908*(1-Mf)**0.794 ## From Sean's Code
strain_amplitude = ((Mhalf/Mf)**2)/omega**2/sp.cosh((t-t_p)/tau)

## Creating C functions with Nrpy+'s OutCfunction capabilities

At long last, we have obtained sympy-compliant symbolic expressions for the strain amplitude and phase in accordance for implementing the sBOB model. We now use Nrpy+'s OutCfunction function to generate optimized C functions that will be use towards implementing the model.

In [8]:
strainampfunc_desc="Output sBOB strain amplitude"
strainampfunc_name="get_sBOB_strainamplitude"
strainampfunc_params="const double t, double *strain_amplitude"
strainamp_string = outCfunction(
    outfile  = "returnstring", desc=strainampfunc_desc, name=strainampfunc_name,
    params   = strainampfunc_params,
    body     = outputC(strain_amplitude,"*strain_amplitude",filename="returnstring",params="includebraces=False,preindent=1"))

phasefunc_desc="Output sBOB waveform phase"
phasefunc_name="get_sBOB_phase"
phasefunc_params="const double t, double *phase"
phase_string = outCfunction(
    outfile  = "returnstring", desc=phasefunc_desc, name=phasefunc_name,
    params   = phasefunc_params,
    body     = outputC(phase,"*phase",filename="returnstring",params="includebraces=False,preindent=1"),)

## Outputting C expressions into a C header

In order to construct the C implementation of the sBOB model, the ampitude and phase C functions will be placed in a C header file that will be included in the main C module.

To do so, we use Python's file I/O capabilities and write in our header file. 

In [9]:
header_name = "sBOB_funcs.h"
header_caps = "SBOB_FUNCS_H"
include_headers = "#include <stdio.h> \n #include<math.h> \n #include<stdlib.h> \n #include<string.h>\n"
header_text = "/* \n* Header file for sBOB waveform functions \n* Author: Siddharth Mahesh \n* Standard order of input for all functions is as follows: \n* time, QNM frequency, QNM damping time, reference frequency, peak strain \n* This module was generated using Nrpy+ \n*/\n"
func_names_to_define = [strainampfunc_name,phasefunc_name]
func_args_to_define = [strainampfunc_params,phasefunc_params]
outfile = open(header_name,"w")
outfile.write(header_text)
outfile.write(include_headers)
outfile.write("#ifndef {hcaps}\n".format(hcaps = header_caps))
outfile.write("#define {hcaps}\n".format(hcaps = header_caps))
for i in range(len(func_names_to_define)):
    outfile.write("void {funcname}({funcparams});\n".format(funcname = func_names_to_define[i],funcparams=func_args_to_define[i]))
outfile.write("#endif\n")
outfile.write(strainamp_string)
outfile.write(phase_string)
outfile.close()
